# About this notebook

@author: Yingding Wang\
@created: 28.Dec 2023\
@upated: 28.Dec 2023\
@version: 1

This notebook demonstrate the use of sqlalchemy client to connect to a postgres db.

* postgres client (psycopg): https://www.psycopg.org/
* Documentation (psycopg): https://www.psycopg.org/docs/usage.html
* Sqlalchemy: https://docs.sqlalchemy.org/en/20/
* use sqlalchemy with psycopg (3): https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#module-sqlalchemy.dialects.postgresql.psycopg
* https://stackoverflow.com/questions/73596058/creating-an-sqlalchemy-engine-based-on-psycopg3/75885662#75885662

In [1]:
import sys, os
# !{sys.executable} -m pip install --upgrade --use-feature=2020-resolver psycopg2-binary 2.9.9 python-dotenv==1.0.0
!{sys.executable} -m pip install --user --upgrade psycopg[binary]==3.1.16 SQLAlchemy==2.0.24 python-dotenv==1.0.0

## (Option 1) Edit postgres.env file for DB connection

In [2]:
# ENV_FILE="postgres.env"

### Uncomment the following cell and edit with your PostresDB credential to connect to DB

In [3]:
'''
%%writefile $ENV_FILE
# environment variables for Postgres DB 14.1 
DB_HOST="Database_Host_DNS_NAME"
DB_NAME="Database_Name"
DB_PORT="Database_Port"
DB_USER="Database_Login_User_Name"
DB_USER_PW="Database_Login_Password"
'''

'\n%%writefile $ENV_FILE\n# environment variables for Postgres DB 14.1 \nDB_HOST="Database_Host_DNS_NAME"\nDB_NAME="Database_Name"\nDB_PORT="Database_Port"\nDB_USER="Database_Login_User_Name"\nDB_USER_PW="Database_Login_Password"\n'

In [4]:
# from dotenv import load_dotenv
# load_dotenv(dotenv_path="postgres.env", override=True)

"""
print(f"\
{os.environ['DB_HOST']}\n\
{os.environ['DB_NAME']}\n\
{os.environ['DB_PORT']}\n\
{os.environ['DB_USER']}\n\
{os.environ['DB_USER_PW']}\n\
")
"""

'\nprint(f"{os.environ[\'DB_HOST\']}\n{os.environ[\'DB_NAME\']}\n{os.environ[\'DB_PORT\']}\n{os.environ[\'DB_USER\']}\n{os.environ[\'DB_USER_PW\']}\n")\n'

In [5]:
# db_conf_dict = {
#     "dbname":  os.environ['DB_NAME'],
#     "user":    os.environ['DB_USER'],
#     "password":os.environ['DB_USER_PW'],
#     "host":    os.environ['DB_HOST'],
#     "port": os.environ['DB_PORT']  
# }
# print(db_conf_dict)

## (Option 2) Loading credentials from ENV variables
The env variables can be injected from Kubeflow PodDefault

In [6]:
'''Uncomment this cell if you are using opton1'''
from dataclasses import dataclass
import sqlalchemy
import psycopg

@dataclass
class DBConfig:   
    # DB_HOST: str = ""
    # DB_USER: str = ""
    # DB_USER_PW: str = ""
    # DB_NAME: str = ""
    # DB_SCHEMA: str = ""
    # DB_STAGING_PORT: int = 5432
    # DB_ANALYTICS_PORT: int = 5432
    

    def __post_init__(self):
        self.DB_HOST = self._load_env("SCIVIAS_DB_HOST")
        self.DB_USER = self._load_env("SCIVIAS_DB_USERNAME")
        self.DB_USER_PW = self._load_env("SCIVIAS_DB_PASSWORD")
        self.DB_NAME = self._load_env("SCIVIAS_DB_NAME")
        self.DB_SCHEMA = self._load_env("SCIVIAS_DB_SCHEMA")
        self.DB_STAGING_PORT = self._load_env("SCIVIAS_STAGING_DB_PORT", 5432)
        self.DB_ANALYTICS_PORT = self._load_env("SCIVIAS_ANALYTICS_DB_PORT", 5432)
    
    
    @classmethod
    def _load_env(clz, key: str, default = ""):
        return os.environ.get(key, default)
    
    
    def __repr__(self) -> str:
        """for program and developer obj representation also used by jupyter notebook cell"""
        return f"{self.DB_HOST}\n{self.DB_USER}\n{self.DB_USER_PW}\n{self.DB_NAME}\n{self.DB_SCHEMA}\n{self.DB_STAGING_PORT}\n{self.DB_ANALYTICS_PORT}"
        
    def __str__(self):
        """for user print() function"""
        return self.__repr__()
    
    
    def to_psycopg_dic(self, is_analytics: True) -> dict:
        return {
            "dbname":  self.DB_NAME,
            "user":    self.DB_USER,
            "password": self.DB_USER_PW,
            "host":     self.DB_HOST,
            "port":  self.DB_ANALYTICS_PORT if is_analytics else self.DB_STAGING_PORT,   
        }
    
    
    def to_sqlalchemy_url(self, is_analytics: True) -> sqlalchemy.engine.url.URL:
        return sqlalchemy.URL.create(
            drivername="postgresql+psycopg",
            username=self.DB_USER,
            password=self.DB_USER_PW,  # plain (unescaped) text
            host=self.DB_HOST,
            database=self.DB_NAME,
            port=self.DB_ANALYTICS_PORT if is_analytics else self.DB_STAGING_PORT,
        )
        

    
db_conf_obj = DBConfig()
# db_conf_dict = db_conf_obj.to_psycopg_dic(is_analytics=True)
sqlalchemy_url_obj = db_conf_obj.to_sqlalchemy_url(is_analytics=True) 

## Execute SQL with sqlalchemy
* https://docs.sqlalchemy.org/en/20/core/engines.html
* https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#module-sqlalchemy.dialects.postgresql.psycopg

```python
# engine = create_engine('postgresql+psycopg://', creator=connector)
sync_engine = sqlalchemy.create_engine("postgresql+psycopg://", scott:tiger@localhost/test")
```

In [7]:
# create sync engine with url_object
sync_engine = sqlalchemy.create_engine(sqlalchemy_url_obj)

In [8]:
# create table with name test if not exit
# sql1="CREATE TABLE IF NOT EXISTS test (id serial PRIMARY KEY, num integer, data varchar);"

In [9]:
# run_sql(config, sql1)

## Run SQL with results
https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html

In [10]:
from typing import List, Tuple
from collections.abc import Iterable

def print_tuple_list(it: Iterable) -> None:
    for el in it:
        print(*el, sep=",") # print tuple
        

# def run_sql_with_return(engine, sql) -> sqlalchemy.engine.cursor.CursorResult:
def run_sql_with_return(engine, sql) -> Iterable:    
    with engine.connect() as con:
        rs = con.execute(sqlalchemy.text(sql))
        # for row in rs:
        #     print(row)
        return rs

In [11]:
# https://stackoverflow.com/a/24008869
# sql_show_tables="select relname from pg_class;"
sql_show_tables="select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';"

In [12]:
result = run_sql_with_return(sync_engine, sql_show_tables)
print_tuple_list(result)

lab_data


In [13]:
result = run_sql_with_return(sync_engine, "select count(*) from lab_data")
print_tuple_list(result)

236884


### Postgres Upsert
https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#postgresql-insert-on-conflict